In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import pandas as pd
import time, os, copy

In [11]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'tiny-imagenet-5'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

use_gpu = torch.cuda.is_available()

In [12]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=50):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for data in dataloaders[phase]:
                # get the inputs
                inputs, labels = data

                # wrap them in Variable
                if use_gpu:
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data[0] * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

# Resnet 34 - 50 epochs / step size change

In [16]:
# Load pre-trained model and reset final fully connected layer

model_ft = models.resnet34(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 5)

if use_gpu:
    model_ft = model_ft.cuda()

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 5 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.1)

# Train Model
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=50)

# Save Model
torch.save(model_ft.state_dict(), "model_ft_resnet34_2")

Epoch 0/49
----------
train Loss: 1.5817 Acc: 0.3896
val Loss: 1.0836 Acc: 0.5760

Epoch 1/49
----------
train Loss: 1.3471 Acc: 0.4816
val Loss: 0.8490 Acc: 0.6720

Epoch 2/49
----------
train Loss: 1.2344 Acc: 0.5204
val Loss: 0.9405 Acc: 0.6560

Epoch 3/49
----------
train Loss: 1.1186 Acc: 0.5720
val Loss: 0.9057 Acc: 0.7160

Epoch 4/49
----------
train Loss: 1.0892 Acc: 0.5764
val Loss: 0.6966 Acc: 0.7520

Epoch 5/49
----------
train Loss: 0.9141 Acc: 0.6460
val Loss: 0.6087 Acc: 0.7880

Epoch 6/49
----------
train Loss: 0.8804 Acc: 0.6548
val Loss: 0.5911 Acc: 0.7720

Epoch 7/49
----------
train Loss: 0.8439 Acc: 0.6692
val Loss: 0.6347 Acc: 0.7880

Epoch 8/49
----------
train Loss: 0.8273 Acc: 0.6852
val Loss: 0.5749 Acc: 0.7920

Epoch 9/49
----------
train Loss: 0.8179 Acc: 0.6904
val Loss: 0.5693 Acc: 0.8040

Epoch 10/49
----------
train Loss: 0.7876 Acc: 0.7116
val Loss: 0.5832 Acc: 0.7880

Epoch 11/49
----------
train Loss: 0.7710 Acc: 0.7108
val Loss: 0.5872 Acc: 0.7720

Ep

In [17]:
model_conv = torchvision.models.resnet34(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 5)

if use_gpu:
    model_conv = model_conv.cuda()

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opoosed to before.
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 5 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=5, gamma=0.1)

# Train Model
model_cnn = train_model(model_conv, criterion, optimizer_conv, exp_lr_scheduler, num_epochs=50)

# Save Model
torch.save(model_cnn.state_dict(), "model_cnn_resnet34_2")

Epoch 0/49
----------
train Loss: 1.4761 Acc: 0.4112
val Loss: 0.7789 Acc: 0.7080

Epoch 1/49
----------
train Loss: 1.3704 Acc: 0.4776
val Loss: 0.9057 Acc: 0.6800

Epoch 2/49
----------
train Loss: 1.4245 Acc: 0.4856
val Loss: 0.8480 Acc: 0.6480

Epoch 3/49
----------
train Loss: 1.3973 Acc: 0.4788
val Loss: 0.9317 Acc: 0.6680

Epoch 4/49
----------
train Loss: 1.3126 Acc: 0.5196
val Loss: 0.9095 Acc: 0.6760

Epoch 5/49
----------
train Loss: 1.1452 Acc: 0.5504
val Loss: 0.8129 Acc: 0.6880

Epoch 6/49
----------
train Loss: 1.1106 Acc: 0.5652
val Loss: 0.8193 Acc: 0.6800

Epoch 7/49
----------
train Loss: 1.1169 Acc: 0.5596
val Loss: 0.8120 Acc: 0.6920

Epoch 8/49
----------
train Loss: 1.0802 Acc: 0.5736
val Loss: 0.7779 Acc: 0.6760

Epoch 9/49
----------
train Loss: 1.0671 Acc: 0.5728
val Loss: 0.7843 Acc: 0.7000

Epoch 10/49
----------
train Loss: 1.0886 Acc: 0.5648
val Loss: 0.7801 Acc: 0.6720

Epoch 11/49
----------
train Loss: 1.0986 Acc: 0.5636
val Loss: 0.7944 Acc: 0.6920

Ep

# Resnet 34 - 50 epochs

In [5]:
# Load pre-trained model and reset final fully connected layer

model_ft = models.resnet34(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 5)

if use_gpu:
    model_ft = model_ft.cuda()

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

# Train Model
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=50)

# Save Model
torch.save(model_ft.state_dict(), "model_ft_resnet34")

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /home/paperspace/.torch/models/resnet34-333f7ec4.pth
100%|██████████| 87306240/87306240 [00:02<00:00, 29992577.12it/s]


Epoch 0/49
----------
train Loss: 1.5903 Acc: 0.3884
val Loss: 1.1877 Acc: 0.5360

Epoch 1/49
----------
train Loss: 1.3835 Acc: 0.4732
val Loss: 0.9116 Acc: 0.6560

Epoch 2/49
----------
train Loss: 1.2411 Acc: 0.5328
val Loss: 0.8065 Acc: 0.7600

Epoch 3/49
----------
train Loss: 1.1321 Acc: 0.5592
val Loss: 0.7790 Acc: 0.7560

Epoch 4/49
----------
train Loss: 1.0991 Acc: 0.5760
val Loss: 0.9162 Acc: 0.7120

Epoch 5/49
----------
train Loss: 1.0753 Acc: 0.5900
val Loss: 0.6682 Acc: 0.7200

Epoch 6/49
----------
train Loss: 1.0198 Acc: 0.6048
val Loss: 0.6580 Acc: 0.7440

Epoch 7/49
----------
train Loss: 0.8710 Acc: 0.6772
val Loss: 0.5510 Acc: 0.7920

Epoch 8/49
----------
train Loss: 0.8283 Acc: 0.6856
val Loss: 0.5467 Acc: 0.8000

Epoch 9/49
----------
train Loss: 0.7963 Acc: 0.7036
val Loss: 0.5492 Acc: 0.8000

Epoch 10/49
----------
train Loss: 0.7745 Acc: 0.7064
val Loss: 0.5338 Acc: 0.7920

Epoch 11/49
----------
train Loss: 0.7811 Acc: 0.7024
val Loss: 0.5755 Acc: 0.7960

Ep

In [6]:
model_conv = torchvision.models.resnet34(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 5)

if use_gpu:
    model_conv = model_conv.cuda()

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opoosed to before.
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

# Train Model
model_cnn = train_model(model_conv, criterion, optimizer_conv, exp_lr_scheduler, num_epochs=50)

# Save Model
torch.save(model_cnn.state_dict(), "model_cnn_resnet34")

Epoch 0/49
----------
train Loss: 1.4628 Acc: 0.4204
val Loss: 0.9111 Acc: 0.6320

Epoch 1/49
----------
train Loss: 1.4071 Acc: 0.4816
val Loss: 0.8568 Acc: 0.6840

Epoch 2/49
----------
train Loss: 1.3690 Acc: 0.4936
val Loss: 0.8589 Acc: 0.6560

Epoch 3/49
----------
train Loss: 1.3394 Acc: 0.4972
val Loss: 0.7464 Acc: 0.7360

Epoch 4/49
----------
train Loss: 1.3182 Acc: 0.5172
val Loss: 0.7755 Acc: 0.7160

Epoch 5/49
----------
train Loss: 1.2841 Acc: 0.5304
val Loss: 0.7919 Acc: 0.7120

Epoch 6/49
----------
train Loss: 1.3524 Acc: 0.5100
val Loss: 0.8561 Acc: 0.6840

Epoch 7/49
----------
train Loss: 1.1615 Acc: 0.5480
val Loss: 0.7625 Acc: 0.7080

Epoch 8/49
----------
train Loss: 1.1455 Acc: 0.5604
val Loss: 0.8268 Acc: 0.7160

Epoch 9/49
----------
train Loss: 1.1116 Acc: 0.5680
val Loss: 0.7763 Acc: 0.6880

Epoch 10/49
----------
train Loss: 1.0848 Acc: 0.5700
val Loss: 0.7956 Acc: 0.7040

Epoch 11/49
----------
train Loss: 1.1157 Acc: 0.5596
val Loss: 0.7914 Acc: 0.7000

Ep

# Resnet 34 - 75 epochs

In [14]:
# Load pre-trained model and reset final fully connected layer

model_ft = models.resnet34(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 5)

if use_gpu:
    model_ft = model_ft.cuda()

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

# Train Model
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=75)

# Save Model
torch.save(model_ft.state_dict(), "model_ft_resnet34_75")

Epoch 0/74
----------
train Loss: 1.5745 Acc: 0.4020
val Loss: 1.4593 Acc: 0.5160

Epoch 1/74
----------
train Loss: 1.3688 Acc: 0.4728
val Loss: 0.8884 Acc: 0.6800

Epoch 2/74
----------
train Loss: 1.2361 Acc: 0.5148
val Loss: 0.9192 Acc: 0.6640

Epoch 3/74
----------
train Loss: 1.1260 Acc: 0.5796
val Loss: 0.6923 Acc: 0.7280

Epoch 4/74
----------
train Loss: 1.1330 Acc: 0.5624
val Loss: 0.7200 Acc: 0.7480

Epoch 5/74
----------
train Loss: 0.9933 Acc: 0.6116
val Loss: 0.7503 Acc: 0.7040

Epoch 6/74
----------
train Loss: 1.0047 Acc: 0.6128
val Loss: 0.8134 Acc: 0.7040

Epoch 7/74
----------
train Loss: 0.8211 Acc: 0.6896
val Loss: 0.6397 Acc: 0.7680

Epoch 8/74
----------
train Loss: 0.8003 Acc: 0.6916
val Loss: 0.6431 Acc: 0.7680

Epoch 9/74
----------
train Loss: 0.7557 Acc: 0.7172
val Loss: 0.6245 Acc: 0.7880

Epoch 10/74
----------
train Loss: 0.7848 Acc: 0.7016
val Loss: 0.6287 Acc: 0.7840

Epoch 11/74
----------
train Loss: 0.7620 Acc: 0.7140
val Loss: 0.6007 Acc: 0.7880

Ep

In [15]:
model_conv = torchvision.models.resnet34(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 5)

if use_gpu:
    model_conv = model_conv.cuda()

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opoosed to before.
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

# Train Model
model_cnn = train_model(model_conv, criterion, optimizer_conv, exp_lr_scheduler, num_epochs=75)

# Save Model
torch.save(model_cnn.state_dict(), "model_cnn_resnet34_75")

Epoch 0/74
----------
train Loss: 1.5266 Acc: 0.4028
val Loss: 1.2208 Acc: 0.4640

Epoch 1/74
----------
train Loss: 1.3668 Acc: 0.4800
val Loss: 0.8056 Acc: 0.7080

Epoch 2/74
----------
train Loss: 1.3596 Acc: 0.4944
val Loss: 0.8967 Acc: 0.7000

Epoch 3/74
----------
train Loss: 1.3380 Acc: 0.5108
val Loss: 0.9398 Acc: 0.6480

Epoch 4/74
----------
train Loss: 1.2737 Acc: 0.5160
val Loss: 0.8356 Acc: 0.6840

Epoch 5/74
----------
train Loss: 1.3876 Acc: 0.4940
val Loss: 0.8904 Acc: 0.6880

Epoch 6/74
----------
train Loss: 1.3530 Acc: 0.5136
val Loss: 0.9550 Acc: 0.6600

Epoch 7/74
----------
train Loss: 1.1371 Acc: 0.5468
val Loss: 0.8007 Acc: 0.7200

Epoch 8/74
----------
train Loss: 1.1346 Acc: 0.5500
val Loss: 0.8100 Acc: 0.7160

Epoch 9/74
----------
train Loss: 1.0943 Acc: 0.5812
val Loss: 0.7990 Acc: 0.6920

Epoch 10/74
----------
train Loss: 1.1118 Acc: 0.5576
val Loss: 0.7970 Acc: 0.6840

Epoch 11/74
----------
train Loss: 1.0959 Acc: 0.5740
val Loss: 0.7927 Acc: 0.7040

Ep

# Resnet 50

In [8]:
# Load pre-trained model and reset final fully connected layer

model_ft = models.resnet50(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 5)

if use_gpu:
    model_ft = model_ft.cuda()

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

# Train Model
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=100)

# Save Model
torch.save(model_ft.state_dict(), "model_ft_resnet50")

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /home/paperspace/.torch/models/resnet50-19c8e357.pth
100%|██████████| 102502400/102502400 [00:02<00:00, 48653000.26it/s]


Epoch 0/99
----------
train Loss: 1.4689 Acc: 0.4488
val Loss: 0.9085 Acc: 0.6720

Epoch 1/99
----------
train Loss: 1.2961 Acc: 0.4800
val Loss: 0.8752 Acc: 0.6600

Epoch 2/99
----------
train Loss: 1.1746 Acc: 0.5340
val Loss: 0.8267 Acc: 0.7120

Epoch 3/99
----------
train Loss: 1.0647 Acc: 0.5932
val Loss: 1.3649 Acc: 0.6560

Epoch 4/99
----------
train Loss: 1.0249 Acc: 0.6040
val Loss: 0.7031 Acc: 0.7560

Epoch 5/99
----------
train Loss: 0.9926 Acc: 0.6176
val Loss: 0.7050 Acc: 0.7120

Epoch 6/99
----------
train Loss: 0.9463 Acc: 0.6376
val Loss: 0.8518 Acc: 0.7040

Epoch 7/99
----------
train Loss: 0.8095 Acc: 0.6944
val Loss: 0.6029 Acc: 0.7920

Epoch 8/99
----------
train Loss: 0.7623 Acc: 0.7116
val Loss: 0.6112 Acc: 0.7840

Epoch 9/99
----------
train Loss: 0.7522 Acc: 0.7152
val Loss: 0.5960 Acc: 0.7960

Epoch 10/99
----------
train Loss: 0.7361 Acc: 0.7328
val Loss: 0.6274 Acc: 0.7800

Epoch 11/99
----------
train Loss: 0.7429 Acc: 0.7192
val Loss: 0.6398 Acc: 0.7720

Ep

train Loss: 0.6808 Acc: 0.7384
val Loss: 0.5933 Acc: 0.7960

Epoch 99/99
----------
train Loss: 0.6543 Acc: 0.7616
val Loss: 0.5850 Acc: 0.7920

Training complete in 70m 6s
Best val Acc: 0.812000


In [13]:
model_conv = torchvision.models.resnet50(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 5)

if use_gpu:
    model_conv = model_conv.cuda()

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opoosed to before.
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

# Train Model
model_cnn = train_model(model_conv, criterion, optimizer_conv, exp_lr_scheduler, num_epochs=100)

# Save Model
torch.save(model_cnn.state_dict(), "model_cnn_resnet50")

Epoch 0/99
----------
train Loss: 1.4926 Acc: 0.3964
val Loss: 0.9286 Acc: 0.6520

Epoch 1/99
----------
train Loss: 1.3984 Acc: 0.4664
val Loss: 0.8819 Acc: 0.6440

Epoch 2/99
----------
train Loss: 1.3993 Acc: 0.4640
val Loss: 0.7224 Acc: 0.7360

Epoch 3/99
----------
train Loss: 1.3145 Acc: 0.4992
val Loss: 0.8241 Acc: 0.7000

Epoch 4/99
----------
train Loss: 1.3343 Acc: 0.4952
val Loss: 0.7570 Acc: 0.7160

Epoch 5/99
----------
train Loss: 1.3166 Acc: 0.4916
val Loss: 0.9656 Acc: 0.6440

Epoch 6/99
----------
train Loss: 1.3307 Acc: 0.5032
val Loss: 0.7986 Acc: 0.6760

Epoch 7/99
----------
train Loss: 1.1205 Acc: 0.5504
val Loss: 0.6932 Acc: 0.7280

Epoch 8/99
----------
train Loss: 1.0801 Acc: 0.5780
val Loss: 0.6995 Acc: 0.7240

Epoch 9/99
----------
train Loss: 1.0782 Acc: 0.5720
val Loss: 0.6989 Acc: 0.7280

Epoch 10/99
----------
train Loss: 1.1147 Acc: 0.5640
val Loss: 0.6999 Acc: 0.7200

Epoch 11/99
----------
train Loss: 1.0535 Acc: 0.5852
val Loss: 0.7130 Acc: 0.7320

Ep

train Loss: 1.0868 Acc: 0.5696
val Loss: 0.6833 Acc: 0.7440

Epoch 99/99
----------
train Loss: 1.0595 Acc: 0.5812
val Loss: 0.6620 Acc: 0.7520

Training complete in 21m 13s
Best val Acc: 0.764000
